In [6]:
import keras
keras.__version__

'1.0.7'

In [1]:
# Oscar Chang's code as the best performer for inclass compeition
# copied from http://llcao.net/cu-deeplearning17/code/oscar.py
# the only change I made is to remove "gamma_regularizer=None, beta_regularizer=None" in the batch normalization step 
#    coz they are not available in old version of Keras
from __future__ import print_function
import numpy as np

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization

batch_size = 128
nb_classes = 10
nb_epoch = 30

def low_gamma_init(shape, name=None):
    #value = np.random.normal(loc=0.0, scale=0.05, size=shape)
    value = 0.2 * np.ones(shape)
    return K.variable(value, name=name)

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train[:3000]
y_train = y_train[:3000]

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(BatchNormalization(epsilon=0.001, mode=2, axis=-1, momentum=0.99, weights=None, beta_init='zero', gamma_init=low_gamma_init))
#          #, gamma_regularizer=None, beta_regularizer=None))
model.add(Activation('relu'))
model.add(BatchNormalization(epsilon=0.001, mode=2, axis=-1, momentum=0.99, weights=None, beta_init='zero', gamma_init=low_gamma_init))
#          #, gamma_regularizer=None, beta_regularizer=None))
model.add(Activation('relu'))

model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

# model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
#           verbose=1, validation_data=(X_test, Y_test))

datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=5,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train,
                                 batch_size=batch_size),
                    samples_per_epoch=X_train.shape[0],
                    nb_epoch=nb_epoch,
                    validation_data=(X_test, Y_test))

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5005)


X_train shape: (3000, 1, 28, 28)
3000 train samples
10000 test samples
Epoch 1/30
3000/3000 [==============================] - 0s - loss: nan - acc: 0.0950 - val_loss: nan - val_acc: 0.0980
Epoch 2/30
3000/3000 [==============================] - 0s - loss: nan - acc: 0.0950 - val_loss: nan - val_acc: 0.0980
Epoch 3/30
3000/3000 [==============================] - 0s - loss: nan - acc: 0.0950 - val_loss: nan - val_acc: 0.0980
Epoch 4/30
3000/3000 [==============================] - 0s - loss: nan - acc: 0.0950 - val_loss: nan - val_acc: 0.0980
Epoch 5/30
3000/3000 [==============================] - 0s - loss: nan - acc: 0.0950 - val_loss: nan - val_acc: 0.0980
Epoch 6/30
3000/3000 [==============================] - 0s - loss: nan - acc: 0.0950 - val_loss: nan - val_acc: 0.0980
Epoch 7/30
3000/3000 [==============================] - 0s - loss: nan - acc: 0.0950 - val_loss: nan - val_acc: 0.0980
Epoch 8/30
3000/3000 [==============================] - 0s - loss: nan - acc: 0.0950 - val_loss:

# similar code without batch normalization

In [4]:
batch_size = 128
nb_classes = 10
nb_epoch = 30

def low_gamma_init(shape, name=None):
    #value = np.random.normal(loc=0.0, scale=0.05, size=shape)
    value = 0.2 * np.ones(shape)
    return K.variable(value, name=name)

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train[:3000]
y_train = y_train[:3000]

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

#model.add(BatchNormalization(epsilon=0.001, mode=2, axis=-1, momentum=0.99, weights=None, beta_init='zero', gamma_init=low_gamma_init))
#          #, gamma_regularizer=None, beta_regularizer=None))
#model.add(Activation('relu'))
#model.add(BatchNormalization(epsilon=0.001, mode=2, axis=-1, momentum=0.99, weights=None, beta_init='zero', gamma_init=low_gamma_init))
#          #, gamma_regularizer=None, beta_regularizer=None))
model.add(Activation('relu'))

model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

# model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
#           verbose=1, validation_data=(X_test, Y_test))

datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=5,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train,
                                 batch_size=batch_size),
                    samples_per_epoch=X_train.shape[0],
                    nb_epoch=nb_epoch,
                    validation_data=(X_test, Y_test), verbose=0)

score = model.evaluate(X_test, Y_test, verbose=0)
print('fitting without batch normalization')
print('Test score:', score[0])
print('Test accuracy:', score[1])

X_train shape: (3000, 1, 28, 28)
3000 train samples
10000 test samples
fitting without batch normalization
Test score: 0.0578371805344
Test accuracy: 0.9812
